In [1]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import pandas as pd
import torchaudio
import codes.utils
from codes.preprocess_data import *

In [2]:
FOLDER_ANNOTATIONS = "datos/"
ANNOTATIONS_FILE = ["train_set.txt","test_set.txt"]
N_FOLDS = 4
MBE_DIR_TR = "features/"+ ANNOTATIONS_FILE[0].split(".")[0]+"/"
MBE_DIR_TE = "features/"+ ANNOTATIONS_FILE[1].split(".")[0]+"/"
SAMPLE_RATE = 44100
LEN_SEC = 300
LEN_SAMPLES = LEN_SEC*SAMPLE_RATE
__class_labels = {
'perro'    : 0,
'rana'     : 1,
'lluvia'   : 2,
'motor'    : 3,
'ave'      : 4
}
N_FFT = 2048
HOP = int(N_FFT/2)
N_MELS = 40

In [3]:
df_train = pd.read_csv(FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[0],sep=" ",
                    names=["filepath","start","end","class_name"])
audio_filenames_train = np.unique(df_train.filepath)

In [4]:
for i,file in enumerate(audio_filenames_train):
    mbe,label = preprocess_data(file,FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[0],SAMPLE_RATE,
                                LEN_SAMPLES,N_FFT,HOP,N_MELS,__class_labels,device="cuda")
    numberfile = file.split("/")[1].split(".")[0].replace("audio","")
    namembe = "{}mbe{}.pt".format(MBE_DIR_TR,numberfile)
    torch.save({"data":mbe,"label":label}, namembe)
    if i==0:
        mean_std = mbe
    else:
        mean_std = torch.cat((mean_std,mbe))

/home/vpoblete/Escritorio/tesis/notebooks/codes/preprocess_data.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_tensor = torch.tensor(label,device=device)


In [5]:
df_test = pd.read_csv(FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[1],sep=" ",
                    names=["filepath","start","end","class_name"])
audio_filenames_test = np.unique(df_test.filepath)

In [6]:
for i,file in enumerate(audio_filenames_test):
    mbe,label = preprocess_data(file,FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[1],SAMPLE_RATE,
                                LEN_SAMPLES,N_FFT,HOP,N_MELS,__class_labels,device="cuda")
    numberfile = file.split("/")[1].split(".")[0].replace("audio","")
    namembe = "{}mbe{}.pt".format(MBE_DIR_TE,numberfile)
    torch.save({"data":mbe,"label":label}, namembe)
    mean_std = torch.cat((mean_std,mbe))

In [7]:
mean = torch.mean(mean_std)
std = torch.std(mean_std)
print(mean,std)

tensor(-8.4723, device='cuda:0') tensor(2.8595, device='cuda:0')


In [8]:
for i,file in enumerate(audio_filenames_train):
    mbe,label = preprocess_data(file,FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[0],SAMPLE_RATE,
                                LEN_SAMPLES,N_FFT,HOP,N_MELS,__class_labels,device="cuda")
    numberfile = file.split("/")[1].split(".")[0].replace("audio","")
    namembe = "{}mbe{}.pt".format(MBE_DIR_TR,numberfile)
    mbe = (mbe-mean)/std
    torch.save({"data":mbe,"label":label}, namembe)
    if i==0:
        mean_std = mbe
    else:
        mean_std = torch.cat((mean_std,mbe))

In [9]:
for i,file in enumerate(audio_filenames_test):
    mbe,label = preprocess_data(file,FOLDER_ANNOTATIONS+ANNOTATIONS_FILE[1],SAMPLE_RATE,
                                LEN_SAMPLES,N_FFT,HOP,N_MELS,__class_labels,device="cuda")
    numberfile = file.split("/")[1].split(".")[0].replace("audio","")
    namembe = "{}mbe{}.pt".format(MBE_DIR_TE,numberfile)
    mbe = (mbe-mean)/std
    torch.save({"data":mbe,"label":label}, namembe)
    mean_std = torch.cat((mean_std,mbe))

In [10]:
mean1 = torch.mean(mean_std)
std1 = torch.std(mean_std)
print(mean1,std1)

tensor(-8.4723, device='cuda:0') tensor(2.8595, device='cuda:0')
